In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# 1.Thu thập dữ liệu

## Dữ liệu sử dụng
Thống kê dữ liệu theo từng ngày các trường hợp mắc bệnh, tử vong, hồi phục từ 22/1/2020 - 20/6/2021
<br>
Các dữ liệu đều được lấy từ kho dữ liệu COVID-19 của Trung tâm Khoa học và Kỹ thuật Hệ thống (CSSE) thuộc Đại học Johns Hopkins (USA)
<br>
Đường link lấy dữ liệu: https://github.com/CSSEGISandData/COVID-19
<br>
Điều kiện để được sử dụng dữ liệu: Trích dẫn 1 đoạn mà người public dữ liệu yêu cầu (Đã trích dẫn ở cuối bài làm)
<br>
Người ta thu thập dữ liệu bằng cách lấy thông tin từ 1 số lượng lớn các trang web. Đa số thì sẽ lấy thông tin từ từng Quốc Gia

In [2]:
confirm_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recover_df = pd.read_csv('time_series_covid19_recovered_global.csv')
confirm_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,87716,88740,89861,91458,93272,93288,96531,98734,98734,98734
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132437,132449,132459,132461,132469,132476,132481,132484,132488,132490
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,132727,133070,133388,133742,134115,134458,134840,135219,135586,135821
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13813,13813,13813,13826,13828,13836,13839,13842,13842,13842
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,36455,36600,36705,36790,36921,37094,37289,37467,37604,37678


## Tiền xử lý
Gom các cột ngày tháng vào 1 cột có tên là Date, tạo ra 1 bảng duy nhất thể hiện tổng số ca mắc, hồi phục và tử vong của các nước theo ngày

In [3]:
dates = confirm_df.iloc[:,4:]
#Không lấy các cột lat và long vì chúng ta không có dự định biểu thị vị trí trên bản đồ thế giới
confirm_df_melt = confirm_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Confirmed')
death_df_melt = death_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Deaths')
recover_df_melt = recover_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Recovered')

In [4]:
#Ghép các giá trị số ca đã xác nhận, số ca tử vong và số ca hồi phục vào dataframe
covid19_df = pd.merge(left=confirm_df_melt, right=death_df_melt, how='left', on=['Province/State', 'Country/Region', 'Date'])
covid19_df = pd.merge(left=covid19_df, right=recover_df_melt, how='left', on=['Province/State', 'Country/Region', 'Date'])
covid19_df.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,1/22/20,0,0,0.0
1,NaN,Albania,1/22/20,0,0,0.0
2,NaN,Algeria,1/22/20,0,0,0.0
3,NaN,Andorra,1/22/20,0,0,0.0
4,NaN,Angola,1/22/20,0,0,0.0


# 2.Khám phá dữ liệu

In [5]:
covid19_df.shape

(143448, 6)

In [6]:
have_duplicated_rows = covid19_df.duplicated().any()
have_duplicated_rows

False

## Số dòng, cột, ý nghĩa của từng dòng và kiểm tra việc lặp
<br>
Dữ liêu có 142336 dòng và 6 cột. Mỗi dòng biểu thị số ca mắc, số ca tử vong và số ca hồi phục tổng đã được xác nhận tại thời điểm thu thập dữ liệu tại Quốc gia (+ Địa điểm) đó
<br>
Dữ liệu mỗi dòng đều có ý nghĩa giống nhau và không dòng nào bị lặp

## Chú thích dữ liệu (Ý nghĩa của từng cột)
__Province_State__: Tên tiểu bang, địa danh,...
<br>
__Country_Region__: Tên quốc gia
<br>
__Date__: Thời gian thu thập dữ liệu
<br>
__Confirmed__: Số ca mắc bệnh đã xác nhận
<br>
__Deaths__: Số ca tử vong đã được xác nhận
<br>
__Recovered__: Số ca hồi phục đã được xác nhận

In [7]:
covid19_df.dtypes

Province/State     object
Country/Region     object
Date               object
Confirmed           int64
Deaths              int64
Recovered         float64
dtype: object

## Kiểu dữ liệu
Cột __Province_State__, __Country_Region__, __Date__ có kiểu dữ liệu object cần xử lý tiếp tục
<br>
Cột __Confirmed__, __Deaths__, __Recovered__ có kiểu dữ liệu numeric. Để đồng hộ hoá thì chúng ta sẽ chuyển kiểu dữ liệu của __Recovered__ từ float64 về int64 như 2 cái trước.

In [8]:
def open_object_dtype(s):
    dtypes = set(s.apply(lambda x: type(x)).unique())
    return dtypes
print(open_object_dtype(covid19_df['Province/State']))
print(open_object_dtype(covid19_df['Country/Region']))
print(open_object_dtype(covid19_df['Date']))

{<class 'str'>, <class 'float'>}
{<class 'str'>}
{<class 'str'>}


In [9]:
covid19_df['Province/State'].unique()

array([nan, 'Australian Capital Territory', 'New South Wales',
       'Northern Territory', 'Queensland', 'South Australia', 'Tasmania',
       'Victoria', 'Western Australia', 'Alberta', 'British Columbia',
       'Diamond Princess', 'Grand Princess', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Northwest Territories',
       'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island',
       'Quebec', 'Repatriated Travellers', 'Saskatchewan', 'Yukon',
       'Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu',
       'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai',
       'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin',
       'Tibet', 'Unknown', 'Xinjiang', 'Yunnan', 'Zhejiang',
       'Faroe Islands', 'Greenland', 'French Guiana', 'French Polynesia',
       'Guadeloupe', 'Martiniq

In [10]:
covid19_df['Date'] = pd.to_datetime(covid19_df['Date'], format = '%m/%d/%y')
covid19_df['Recovered'] = covid19_df['Recovered'].fillna(0)
covid19_df['Recovered'] = covid19_df['Recovered'].astype('int64')
covid19_df['Province/State'] = covid19_df[['Province/State','Country/Region']].replace(np.nan,'')

In [11]:
num_cols = ['Date','Confirmed','Deaths','Recovered']
def missing_ratio(df):
    return df.isna().mean() * 100
nume_col_profiles_df = covid19_df[num_cols].agg([missing_ratio, 'min', 'max'])
nume_col_profiles_df

,Date,Confirmed,Deaths,Recovered
missing_ratio,0.0,0.0,0.0,0.0
min,2020-01-22 00:00:00,0.0,0.0,0.0
max,2021-06-20 00:00:00,33541887.0,601824.0,28844199.0


## Dữ liệu numeric
<br>
Dữ liệu không có giá trị thiếu
<br>
Dữ liệu không có giá trị bất thường về ngày đầu tiên và ngày cuối cùng thu thập dữ liệu, số ca mắc, tử vong, hồi phục cao nhất tính tới ngày 20/6/2021

In [12]:
cat_cols = list(set(covid19_df.columns) - set(num_cols))
def missing_ratio(df):
    return df.isna().mean() * 100
def num_diff_vals(df):
    return df.nunique()
def diff_vals(df):
    return df.unique()
cate_col_profiles_df = covid19_df[cat_cols].agg([missing_ratio, num_diff_vals, diff_vals])
cate_col_profiles_df

,Province/State,Country/Region
missing_ratio,0.0,0.0
num_diff_vals,88,194
diff_vals,"[, Australian Capital Territory, New South Wal...","[Afghanistan, Albania, Algeria, Andorra, Angol..."


## Dữ liệu categorical
Dữ liệu không có giá trị thiếu
<br>
Dữ liệu có ít quốc gia trên thế giới hơn so với thực tế (194 quốc gia so với 204 quốc gia đã được xác nhận) ngoài ra không còn gì bất thường

# 3. Trả lời câu hỏi

## Câu hỏi
1. Những nước nào có tỉ lệ số người tử vong trên số người hồi phục thấp để chúng ta có thể học tập các biện pháp y tế của họ đồng thời những nước nào có tỉ lệ cao để chúng ta có thể rút kinh nghiệm từ những khó khăn của họ (Chỉ xét những nước có số ca hồi phục lớn hơn 100 và đã có hơn 1000 ca mắc để đảm bảo dữ liệu khách quan).
<br><br>
2. Trong vòng 1 tuần, 1 tháng qua, nước nào có số người nhiễm mới thấp so với thế giới. Từ đó chúng ta có thể học hỏi về cách phòng bệnh và mở cửa giao thương. Đồng thời chúng ta sẽ tìm những nước có người nhiễm cao để rút kinh nghiệm từ những khó khăn của họ và cảnh báo cho người dân không chủ quan.
<br><br>
3. Kiểm tra và thống kê dữ liệu của Việt Nam để chúng ta hiểu rõ hơn tình hình của chúng ta hiện tại để biết khả năng phòng chống và sự phức tạp của dịch bệnh này ở nước ta như thế nào. (Các tỉ lệ so với thế giới, so với trước đó, tốc độ gia tăng,...)

## Tiền xử lý
Để tính số người tử vong trên số người hồi phục, chúng ta phải lấy số ca nhiễm, số ca tử vong và số ca hồi phục mới nhất để tính. Vì vậy chúng ta sẽ tạo 1 dataframe ứng với mỗi quốc gia sẽ là số ca nhiễm, ca hồi phục và ca tử vong mới nhất

In [13]:
country_df = covid19_df[covid19_df['Date']==max(covid19_df['Date'])].reset_index(drop=True).drop('Date', axis=1)
country_df

,Province/State,Country/Region,Confirmed,Deaths,Recovered
0,,Afghanistan,98734,3934,64401
1,,Albania,132490,2454,129918
2,,Algeria,135821,3631,94571
3,,Andorra,13842,127,13650
4,,Angola,37678,859,31676
...,...,...,...,...,...
273,,Vietnam,13258,66,5229
274,,West Bank and Gaza,312673,3551,306240
275,,Yemen,6885,1355,3953
276,,Zambia,129033,1644,108960


Ta thấy có 278 dòng trong khi chúng ta đã khám phá chỉ có 194 quốc gia được thể hiện nên có thể các quốc gia đã bị trùng lặp do sự phân vùng lãnh thổ "Province/State". Chúng ta cần lấy số liệu của từng quốc gia nên chúng ta sẽ nhóm từng quốc gia lại.

In [14]:
country_df = country_df.groupby('Country/Region')[['Confirmed', 'Deaths','Recovered']].sum().reset_index()
country_df

,Country/Region,Confirmed,Deaths,Recovered
0,Afghanistan,98734,3934,64401
1,Albania,132490,2454,129918
2,Algeria,135821,3631,94571
3,Andorra,13842,127,13650
4,Angola,37678,859,31676
...,...,...,...,...
189,Vietnam,13258,66,5229
190,West Bank and Gaza,312673,3551,306240
191,Yemen,6885,1355,3953
192,Zambia,129033,1644,108960


Dữ liệu đã chuẩn xác bây giờ chúng ta sẽ bắt đầu đến phần trả lời câu hỏi

## Câu 1

In [15]:
country_df_1000 = country_df[country_df['Confirmed'] >= 1000]
country_df_1000 = country_df_1000[country_df_1000['Recovered']>=100].reset_index(drop = True)
death_recov_ratio = (country_df_1000['Deaths'] / country_df_1000['Recovered']) * 100
death_recov_df = country_df_1000['Country/Region'].to_frame()
death_recov_df.insert(1,'death_recov_ratio',death_recov_ratio)
death_recov_df

,Country/Region,death_recov_ratio
0,Afghanistan,6.108601
1,Albania,1.888884
2,Algeria,3.839443
3,Andorra,0.930403
4,Angola,2.711832
...,...,...
169,Vietnam,1.262192
170,West Bank and Gaza,1.159548
171,Yemen,34.277764
172,Zambia,1.508811


Tính toán ...

In [16]:
death_recov_df.sort_values('death_recov_ratio',inplace = True)
death_recov_df.reset_index(drop = True)

,Country/Region,death_recov_ratio
0,Singapore,0.054802
1,Bhutan,0.059666
2,Laos,0.154321
3,Timor-Leste,0.249050
4,Qatar,0.267132
...,...,...
169,Yemen,34.277764
170,Taiwan*,48.455428
171,Spain,53.633559
172,Netherlands,66.180308


Sắp xếp theo death_recov_ratio...

In [17]:
top5_most_ratio = death_recov_df[:5]
top5_most_ratio.index = np.arange(1,len(top5_most_ratio)+1) 
top5_most_ratio

,Country/Region,death_recov_ratio
1,Singapore,0.054802
2,Bhutan,0.059666
3,Laos,0.154321
4,Timor-Leste,0.249050
5,Qatar,0.267132


Đây là top 5 những nước có tỉ lệ số ca tử vong trên số ca hồi phục thấp nhất. Có vẻ những nước này có một nền y tế đủ tốt để không bệnh nhân nào không được chữa trị hoặc các nước này có tỉ lệ nhiễm bệnh ít. Đồng thời có vaccine để phòng ngừa dịch bệnh.

In [18]:
top5_worst_ratio = death_recov_df[-5:].sort_values('death_recov_ratio',ascending= False).reset_index(drop = True)
top5_worst_ratio.index = np.arange(1,len(top5_worst_ratio)+1) 
top5_worst_ratio

,Country/Region,death_recov_ratio
1,United Kingdom,821.524664
2,Netherlands,66.180308
3,Spain,53.633559
4,Taiwan*,48.455428
5,Yemen,34.277764


Đây là top 5 những nước có tỉ lệ số ca tử vong trên số ca hồi phục lớn nhất. Những nước này có lẽ có những khó khăn nhất định.

## Câu 2

### Trong 1 tuần

In [19]:
region_df = covid19_df.copy()
week_df = region_df[region_df['Date']==max(region_df['Date']) - timedelta(days = 7)].reset_index(drop=True)\
                    .groupby('Country/Region')[['Confirmed']].sum().reset_index()
infected_num_one_week = country_df['Confirmed'] - week_df['Confirmed']
infected_num_one_week_df = country_df['Country/Region'].to_frame()
infected_num_one_week_df.insert(1,'Infected_num_one_week',infected_num_one_week)
infected_num_one_week_df

,Country/Region,Infected_num_one_week
0,Afghanistan,8873
1,Albania,31
2,Algeria,2433
3,Andorra,29
4,Angola,973
...,...,...
189,Vietnam,2628
190,West Bank and Gaza,1139
191,Yemen,23
192,Zambia,17287


Tính số ca mắc mới của 1 tuần trước...

In [20]:
infected_num_one_week_df.sort_values('Infected_num_one_week', inplace = True)
infected_num_one_week_df = infected_num_one_week_df.reset_index(drop = True)
infected_num_one_week_df

,Country/Region,Infected_num_one_week
0,Palau,0
1,Solomon Islands,0
2,Grenada,0
3,Central African Republic,0
4,San Marino,0
...,...,...
189,Russia,106715
190,Argentina,144599
191,Colombia,191942
192,India,424811


Sắp xếp theo Infected_num_one_week

In [21]:
country_no_week_case_df = infected_num_one_week_df[infected_num_one_week_df['Infected_num_one_week'] == 0]
country_no_week_case_df

,Country/Region,Infected_num_one_week
0,Palau,0
1,Solomon Islands,0
2,Grenada,0
3,Central African Republic,0
4,San Marino,0
5,Holy See,0
6,Samoa,0
7,Iceland,0
8,Kiribati,0
9,Tajikistan,0


Đây là những nước và vùng lãnh thổ không có ca mắc mới nào trong 1 tuần. Chúng ta có thể liên hệ những nước này để học tập. Có vẻ những nước nhỏ thì tình hình dịch bệnh rất ổn định và chính phủ của họ đã kiểm soát được.

In [22]:
top_5_most_infected_one_week_df = infected_num_one_week_df[-5:].sort_values('Infected_num_one_week', ascending = False).reset_index(drop = True)
top_5_most_infected_one_week_df.index = np.arange(1, len(top_5_most_infected_one_week_df) + 1)
top_5_most_infected_one_week_df

,Country/Region,Infected_num_one_week
1,Brazil,515162
2,India,424811
3,Colombia,191942
4,Argentina,144599
5,Russia,106715


Đây là top 5 những nước có số ca mắc mới trong 1 tuần cao nhất. Theo như tôi theo dõi tình hình dịch bệnh thì dạo gần đây, số ca mắc mới của Nga và Ấn Độ liên quan đến biến chủng Delta mới của virus SARS-COV2. Biến chủng này có khả năng lẫy nhiễm rất cao so với chủng bình thường. Còn với các nước Nam Mĩ thì có lẽ do việc tổ chức Copa América có ảnh hưởng lớn đến tình hình dịch bệnh. Đây là nguyên nhân dẫn đến số ca mắc mới của các nước này tăng mạnh trong 1 tuần.

### Trong 1 tháng

In [23]:
month_df = region_df[region_df['Date']==max(region_df['Date']) - timedelta(days = 30)].reset_index(drop=True)
month_df = month_df.groupby('Country/Region')[['Confirmed']].sum().reset_index()
infected_num_one_month = country_df['Confirmed'] - month_df['Confirmed']
infected_num_one_month_df = country_df['Country/Region'].to_frame()
infected_num_one_month_df.insert(1,'Infected_num_one_month',infected_num_one_month)
infected_num_one_month_df

,Country/Region,Infected_num_one_month
0,Afghanistan,33654
1,Albania,337
2,Algeria,9387
3,Andorra,273
4,Angola,5769
...,...,...
189,Vietnam,8317
190,West Bank and Gaza,7705
191,Yemen,253
192,Zambia,36113


Tính số ca mắc mới của 1 tháng trước...

In [24]:
infected_num_one_month_df.sort_values('Infected_num_one_month', inplace = True)
infected_num_one_month_df = infected_num_one_month_df.reset_index(drop = True)
infected_num_one_month_df

,Country/Region,Infected_num_one_month
0,Kiribati,0
1,Samoa,0
2,Micronesia,0
3,Grenada,0
4,Diamond Princess,0
...,...,...
189,US,453395
190,Colombia,753116
191,Argentina,786277
192,Brazil,1956979


Sắp xếp theo Infected_num_one_month

In [25]:
top_5_most_infected_one_month_df = infected_num_one_month_df[-5:].sort_values('Infected_num_one_month', ascending = False).reset_index(drop = True)
top_5_most_infected_one_month_df.index = np.arange(1, len(top_5_most_infected_one_month_df) + 1)
top_5_most_infected_one_month_df

,Country/Region,Infected_num_one_month
1,India,3645931
2,Brazil,1956979
3,Argentina,786277
4,Colombia,753116
5,US,453395


Theo như tôi đã đề cập ở trên, biến chủng Delta mới của Virus SARS-COV2 có khả năng lây nhiễm rất cao so với chủng bình thường.
Còn các nước Nam Mĩ do tổ chức Copa América dẫn đến tăng số ca nhiễm mới trong 1 tháng. Còn ở Hoa Kì, số ca tăng mạnh trong 1 tháng chủ yếu là vì có nhiều người trẻ xét nghiệm dương tính, đặc biệt là ở vùng Nam và Tây nước Mỹ, theo TS Robert Redfield, người đứng đầu CDC. 

In [26]:
country_no_month_case_df = infected_num_one_month_df[infected_num_one_month_df['Infected_num_one_month'] == 0]
country_no_month_case_df

,Country/Region,Infected_num_one_month
0,Kiribati,0
1,Samoa,0
2,Micronesia,0
3,Grenada,0
4,Diamond Princess,0
5,Solomon Islands,0
6,Holy See,0
7,Tajikistan,0
8,Tanzania,0
9,Marshall Islands,0


Đây là những nước và vùng lãnh thổ không có ca mắc mới trong vòng 1 tháng. Chính phủ của các nước và vùng lãnh thổ này có vẻ đã kiểm soát được dịch bệnh.

## Câu 3

# 4.Nhìn lại quá trình làm đồ án

# 5.Tài liệu tham khảo

Tài liệu pandas: https://pandas.pydata.org/docs/reference/index.html
<br>
Tài liệu về covid-19 của trường Đại học John Hopkins: https://github.com/CSSEGISandData/COVID-19
<br>
Tài liệu vẽ matplotlib: https://matplotlib.org/stable/api/index.html
<br>
Nguồn tham khảo 1 số vấn đề gặp phải khi làm đồ án: https://stackoverflow.com/

Dong E, Du H, Gardner L. An interactive web-based dashboard to track COVID-19 in real time. Lancet Inf Dis. 20(5):533-534. doi: 10.1016/S1473-3099(20)30120-1